# Task 1

In [0]:
from pyspark.sql.functions import concat, col, lit 

In [0]:
dbfs_fileStore_prefix = "/FileStore/tables"
prefix = "ontimeperformance"
size = "small"


In [0]:
def clean_column_names(df):
  tempList = [] #Edit01
  for col in df.columns:
      new_name = col.strip()
      new_name = "".join(new_name.split())
      new_name = new_name.replace('.','') 
      tempList.append(new_name) 

  return df.toDF(*tempList) 



In [0]:
def task_1(spark_session, flights_path, aircrafts_path):
    aircrafts_df = spark_session.read.csv(aircrafts_path,
                                        inferSchema=True,
                                        header=True)
    aircrafts_df = aircrafts_df.dropna()
  
    aircrafts = clean_column_names(aircrafts_df)
    
    flights_df = spark_session.read.csv(flights_path,
                                        inferSchema=True,
                                        header=True)
    flights = clean_column_names(flights_df).dropna()
    
    new_flights = flights.join(aircrafts, flights.tail_number == aircrafts.tailnum, how='inner')
    
  

    
    cessna = new_flights.filter(new_flights.manufacturer == "CESSNA") \
                    .orderBy(new_flights.model).groupBy(new_flights.model).count()
    
    cessna = cessna.orderBy('count', ascending=False)
    cessna = cessna.select("*", concat(lit('Cessna '),col("model"),lit(' \t '),col("count")).alias("Result"))
    cLs = cessna.select('Result').take(3)
    for i in cLs:
      print(i[0])
#     haven't change model name
#     not input as a file

    

    

In [0]:
task_1(spark, f"{dbfs_fileStore_prefix}/{prefix}_flights_{size}.csv", f"{dbfs_fileStore_prefix}/{prefix}_aircrafts.csv")

Cessna 172M 55
Cessna 421C 45
Cessna 210-5(205) 28